In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Places Dataset Enriched Filled.csv")

In [3]:
df.head()

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews,location_info,activities,Geographical,Historical,Religious,Natural,Entertainment,Accommodation,Shopping,Food
0,Arugam Bay Beach,6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.800000,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...,"Arugam Bay Beach, located on Sri Lanka's south...","['Surfing', 'Wildlife Watching', 'Yoga Retreat...","Located on the eastern coast of Sri Lanka, Aru...",Arugam Bay has a history linked to the traditi...,"The area is home to several temples, with near...","Arugam Bay features beautiful sandy beaches, l...",Popular activities include beach parties and c...,Arugam Bay offers a range of accommodations fr...,"Local markets and small shops offer souvenirs,...",The area is known for fresh seafood and tradit...
1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.600000,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...,"Mirissa Beach, located on Sri Lanka's southern...","['Whale watching', 'Snorkeling and diving', 'S...",Mirissa Beach is located on the southern coast...,Mirissa has a rich maritime history and has be...,The area is close to the significant Buddhist ...,The beach is known for its stunning natural be...,Mirissa is a popular spot for nightlife activi...,Various types of accommodations are available ...,"Local markets and shops offer handicrafts, bea...",Mirissa offers a variety of seafood options an...
2,Weligama Beach (surf and stay),5.972486,80.435714,"Weligama, Sri Lanka",4.400000,325.0,['Weligama Beach is a fantastic spot for both ...,"Weligama Beach, located on Sri Lanka's souther...","['Surfing', 'Yoga classes', 'Whale watching', ...",Weligama Beach is located on the southern coas...,The area has a rich history as a fishing villa...,"Nearby temples, such as the Parrot Rock Temple...",Weligama Beach is known for its sublime sunset...,"The beach offers lively activities, including ...",Accommodation options range from guesthouses t...,Local markets and shops offer a variety of han...,"Seafood is abundant along the shore, with offe..."
3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",4.459437,582.5,['Ahangama was a bit disappointing for me as a...,"Ahangama, located on the southern coast of Sri...","['Surfing at Ahangama Beach', 'Exploring the K...",Ahangama is located on the southern coast of S...,Ahangama has a rich history connected to the c...,The town and surrounding areas have several Bu...,"Ahangama boasts beautiful beaches, coral reefs...",Ahangama is known for its vibrant surf culture...,"The area has a variety of guesthouses, villas,...","Local artisan shops sell handmade crafts, surf...",Ahangama features traditional Sri Lankan cuisi...
4,Hikkaduwa Beach,6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.700000,1438.0,['Hikkaduwa Beach is a delightful escape for s...,"Hikkaduwa Beach, located on the southwestern c...","['Snorkeling', 'Surfing', 'Glass Bottom Boat T...",Hikkaduwa is located on the southwestern coast...,Hikkaduwa has a rich history as a prominent po...,"The area is home to several Buddhist temples, ...",Hikkaduwa boasts a stunning marine environment...,Hikkaduwa is known for its lively beach scene ...,"Due to its popularity, Hikkaduwa offers a vari...",The area has a range of shops selling local cr...,Hikkaduwa has a vibrant food scene where you c...


In [4]:
# prompt: show the row where lat is null or lng is null

null_lat_lng = df[df['lat'].isnull() | df['lng'].isnull()]
print(null_lat_lng['name'])


89    Leisure World
Name: name, dtype: object


In [5]:
lat = 6.92328
lng = 80.10568

df.loc[df['name'] == 'Leisure World', 'lat'] = lat
df.loc[df['name'] == 'Leisure World', 'lng'] = lng

In [7]:
import pandas as pd
import numpy as np
import spacy
import re
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
from keybert import KeyBERT
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm
from geopy.distance import geodesic
from datetime import datetime
import json

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
import spacy.cli
spacy.cli.download("en_core_web_md")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
nlp = spacy.load("en_core_web_md")

In [10]:
SRI_LANKAN_HUBS = {
    'Colombo': (6.9271, 79.8612),
    'Kandy': (7.2906, 80.6337),
    'Galle': (6.0535, 80.2210),
    'Jaffna': (9.6615, 80.0255),
    'Trincomalee': (8.5874, 81.2152),
    'Sigiriya' : (7.956944, 80.759720),
    'Ella' : (6.8756, 81.043),
    'Pollonnaruwa' : (7.9912, 81.0272),
}

In [11]:
sentence_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
def preprocess_text(text):

    text = str(text)

    text = text.lower()

    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    text = re.sub(r'\S+@\S+', '', text)


    text = re.sub(r'[^\u0d80-\u0dffa-zA-Z0-9\s]', '', text)


    doc = nlp(text)

    processed_tokens = [
        token.lemma_ for token in doc
        if not (
            token.is_stop or token.is_punct or token.like_num or token.ent_type_ in ['DATE', 'TIME', 'MONEY', 'QUANTITY']
            or not (token.is_alpha or '\u0d80' <= token.text <= '\u0dff')
            or token.like_url or token.like_email
        )
    ]

    return ' '.join(processed_tokens)

In [13]:
word_list = []

for word in list(df['name']):
  pp_word = preprocess_text(word)
  word_list.append(pp_word)

In [14]:
len(word_list)

411

In [15]:
SRI_LANKAN_TERMS = word_list

In [16]:
def extract_sri_lankan_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC', 'ORG']]
    sri_lankan_entities = [ent for ent in entities if ent in SRI_LANKAN_TERMS]
    return ', '.join(sri_lankan_entities)

In [17]:
kw_model = KeyBERT('all-MiniLM-L6-v2')

def extract_keywords_keybert(text):
    # Extract keywords using KeyBERT
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3), top_n=10, stop_words='english')

    keywords_list = [keyword[0] for keyword in keywords]
    return ', '.join(keywords_list)

In [18]:
def get_sentiment_score(text):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(text)['compound']

In [19]:
def calculate_distances(lat, lon):
    distances = {}
    for hub, coords in SRI_LANKAN_HUBS.items():
        distances[f'distance_to_{hub}'] = geodesic((lat, lon), coords).kilometers
    return distances

In [20]:
# def get_season(date):
#     month = date.month
#     if month in [12, 1, 2]:
#         return 'Winter'
#     elif month in [3, 4]:
#         return 'First Inter-Monsoon'
#     elif month in [5, 6, 7, 8, 9]:
#         return 'Southwest Monsoon'
#     else:
#         return 'Second Inter-Monsoon'

In [21]:
def extract_features(df, text_columns):
    print("Starting feature extraction...")

    # Preprocess text columns
    for col in text_columns:
        # Skip null values when applying text processing functions
        df[f'{col}_processed'] = df[col].apply(lambda x: preprocess_text(x) if pd.notnull(x) else x)
        df[f'{col}_entities'] = df[col].apply(lambda x: extract_sri_lankan_entities(x) if pd.notnull(x) else x)
        df[f'{col}_keywords'] = df[col].apply(lambda x: extract_keywords_keybert(x) if pd.notnull(x) else x)
        df[f'{col}_sentiment'] = df[col].apply(lambda x: get_sentiment_score(x) if pd.notnull(x) else x)

    # Combine all processed text columns, skipping null values
    df['combined_text'] = df[[f'{col}_processed' for col in text_columns]].apply(
        lambda row: ' '.join(str(val) for val in row if pd.notnull(val)), axis=1
    )

    # TF-IDF Vectorization with n-grams (no change needed here, as TfidfVectorizer handles null values)
    tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
    tfidf_matrix = tfidf.fit_transform(df['combined_text'])

    # Dimensionality Reduction for TF-IDF (no change needed)
    svd = TruncatedSVD(n_components=100)
    reduced_tfidf = svd.fit_transform(tfidf_matrix)

    # Word2Vec embeddings
    # Filter out null values before creating sentences
    sentences = [text.split() for text in df['combined_text'] if pd.notnull(text)]
    word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

    def get_doc_vector(text):
        if pd.isnull(text):
            return np.zeros(100)
        words = text.split()
        word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
        return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(100)

    word2vec_features = np.array([get_doc_vector(text) for text in df['combined_text']])

    # Generate sentence embeddings
    print("Generating sentence embeddings...")
    # Filter out null values before encoding
    valid_texts = [text for text in df['combined_text'] if pd.notnull(text)]
    sentence_embeddings = sentence_model.encode(valid_texts, show_progress_bar=True, batch_size=32)

    # Pad sentence embeddings for null values
    padded_sentence_embeddings = np.zeros((len(df), sentence_embeddings.shape[1]))
    valid_indices = df['combined_text'].notna()
    padded_sentence_embeddings[valid_indices] = sentence_embeddings

    # Add distance features (assuming calculate_distances can handle null values)
    distance_features = df.apply(lambda row: calculate_distances(row['lat'], row['lng']), axis=1)
    df_encoded = pd.concat([df, pd.DataFrame(distance_features.tolist())], axis=1)

    # Combine all features
    feature_df = pd.concat([
        df_encoded,
        pd.DataFrame(reduced_tfidf, columns=[f'tfidf_feature_{i}' for i in range(100)]),
        pd.DataFrame(word2vec_features, columns=[f'word2vec_feature_{i}' for i in range(100)]),
        pd.DataFrame(padded_sentence_embeddings, columns=[f'sent_emb_{i}' for i in range(padded_sentence_embeddings.shape[1])])
    ], axis=1)

    print("Feature extraction completed.")
    return {
        'feature_df': feature_df,
        'tfidf_vectorizer': tfidf,
        'tfidf_matrix': tfidf_matrix,
        'svd_model': svd,
        'word2vec_model': word2vec_model,
        'sentence_embeddings': sentence_embeddings,
        'valid_indices': valid_indices
    }

In [22]:
def get_svd_model(n_components=100):
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    return svd

In [23]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sentence_transformers import SentenceTransformer
from transformers import BertModel, BertTokenizer
import torch


bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # BERT tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')  # Pre-trained BERT model


def get_bert_embedding(text):
    inputs = bert_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

def get_sentence_embedding(text):
    return sentence_model.encode(text)

def preprocess_user_preferences(bucket_list, preferred_activities, feature_extraction_results):
    print("Starting user preferences preprocessing...")

    # Unpack the feature extraction results
    tfidf = feature_extraction_results['tfidf_vectorizer']
    svd = feature_extraction_results['svd_model']
    word2vec_model = feature_extraction_results['word2vec_model']

    # Preprocess and combine user preferences
    processed_preferences = [preprocess_text(item) for item in bucket_list + preferred_activities]
    combined_preferences = ' '.join(processed_preferences)

    user_features = {}

    # TF-IDF features
    user_tfidf = tfidf.transform([combined_preferences])
    user_tfidf_reduced = svd.transform(user_tfidf)
    user_features.update({f'tfidf_feature_{i}': user_tfidf_reduced[0][i] for i in range(100)})

    # Word2Vec features
    def get_doc_vector(text, model):
        words = text.split()
        word_vectors = [model.wv[word] for word in words if word in model.wv]
        return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(100)

    user_word2vec = get_doc_vector(combined_preferences, word2vec_model)
    user_features.update({f'word2vec_feature_{i}': user_word2vec[i] for i in range(100)})

    # Sentence embedding features
    user_sent_emb = get_sentence_embedding(combined_preferences)
    user_features.update({f'sent_emb_{i}': user_sent_emb[i] for i in range(len(user_sent_emb))})

    # BERT features (optional, as it wasn't in the original extraction function)
    user_bert = get_bert_embedding(combined_preferences)
    user_features.update({f'bert_feature_{i}': user_bert[0][i] for i in range(user_bert.shape[1])})

    # Create user DataFrame
    user_df = pd.DataFrame([user_features])

    # Extract entities, keywords, and sentiment (assuming these functions are available)
    user_df['entities'] = extract_sri_lankan_entities(combined_preferences)
    user_df['keywords'] = extract_keywords_keybert(combined_preferences)
    user_df['sentiment'] = get_sentiment_score(combined_preferences)

    print("User preferences preprocessing completed.")
    return user_df

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                411 non-null    object 
 1   lat                 411 non-null    float64
 2   lng                 411 non-null    float64
 3   formatted_address   411 non-null    object 
 4   rating              411 non-null    float64
 5   user_ratings_total  411 non-null    float64
 6   latest_reviews      411 non-null    object 
 7   location_info       411 non-null    object 
 8   activities          411 non-null    object 
 9   Geographical        411 non-null    object 
 10  Historical          410 non-null    object 
 11  Religious           400 non-null    object 
 12  Natural             409 non-null    object 
 13  Entertainment       406 non-null    object 
 14  Accommodation       349 non-null    object 
 15  Shopping            365 non-null    object 
 16  Food    

In [25]:
text_columns = ['latest_reviews', 'location_info', 'activities', 'Geographical', 'Historical', 'Religious', 'Natural', 'Entertainment', 'Accommodation', 'Shopping', 'Food']


feature_extraction_results = extract_features(df, text_columns)

Starting feature extraction...
Generating sentence embeddings...


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Feature extraction completed.


In [27]:
feature_extraction_results['feature_df'].head()

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews,location_info,activities,Geographical,...,sent_emb_374,sent_emb_375,sent_emb_376,sent_emb_377,sent_emb_378,sent_emb_379,sent_emb_380,sent_emb_381,sent_emb_382,sent_emb_383
0,Arugam Bay Beach,6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.800000,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...,"Arugam Bay Beach, located on Sri Lanka's south...","['Surfing', 'Wildlife Watching', 'Yoga Retreat...","Located on the eastern coast of Sri Lanka, Aru...",...,-0.016938,0.040571,-0.030291,-0.012815,0.076728,0.065748,0.124831,-0.009288,-0.083786,-0.013731
1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.600000,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...,"Mirissa Beach, located on Sri Lanka's southern...","['Whale watching', 'Snorkeling and diving', 'S...",Mirissa Beach is located on the southern coast...,...,-0.018548,0.108802,-0.026964,0.034469,0.053808,0.063088,0.061342,-0.023564,-0.095340,-0.061003
2,Weligama Beach (surf and stay),5.972486,80.435714,"Weligama, Sri Lanka",4.400000,325.0,['Weligama Beach is a fantastic spot for both ...,"Weligama Beach, located on Sri Lanka's souther...","['Surfing', 'Yoga classes', 'Whale watching', ...",Weligama Beach is located on the southern coas...,...,0.025448,0.095387,-0.031511,0.015640,0.005926,0.027698,0.089131,0.032127,-0.095310,-0.061990
3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",4.459437,582.5,['Ahangama was a bit disappointing for me as a...,"Ahangama, located on the southern coast of Sri...","['Surfing at Ahangama Beach', 'Exploring the K...",Ahangama is located on the southern coast of S...,...,0.030305,0.063452,0.001474,0.052657,0.018879,0.024863,0.104970,0.008515,-0.095454,-0.041936
4,Hikkaduwa Beach,6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.700000,1438.0,['Hikkaduwa Beach is a delightful escape for s...,"Hikkaduwa Beach, located on the southwestern c...","['Snorkeling', 'Surfing', 'Glass Bottom Boat T...",Hikkaduwa is located on the southwestern coast...,...,0.010923,0.098025,-0.007118,0.077350,0.018785,0.051273,0.063389,0.003538,-0.040687,-0.028157


In [28]:
feature_extraction_results['feature_df'].to_csv('processed_sri_lankan_travel_data_final.csv', index=False)

In [29]:
print(f"Shape of processed destination data: {feature_extraction_results['feature_df'].shape}")
print(f"Columns in processed destination data: {feature_extraction_results['feature_df'].columns.tolist()}")

Shape of processed destination data: (411, 654)
Columns in processed destination data: ['name', 'lat', 'lng', 'formatted_address', 'rating', 'user_ratings_total', 'latest_reviews', 'location_info', 'activities', 'Geographical', 'Historical', 'Religious', 'Natural', 'Entertainment', 'Accommodation', 'Shopping', 'Food', 'latest_reviews_processed', 'latest_reviews_entities', 'latest_reviews_keywords', 'latest_reviews_sentiment', 'location_info_processed', 'location_info_entities', 'location_info_keywords', 'location_info_sentiment', 'activities_processed', 'activities_entities', 'activities_keywords', 'activities_sentiment', 'Geographical_processed', 'Geographical_entities', 'Geographical_keywords', 'Geographical_sentiment', 'Historical_processed', 'Historical_entities', 'Historical_keywords', 'Historical_sentiment', 'Religious_processed', 'Religious_entities', 'Religious_keywords', 'Religious_sentiment', 'Natural_processed', 'Natural_entities', 'Natural_keywords', 'Natural_sentiment', 'E

In [30]:
bucket_list = ["Ancient ruins in Anuradhapura", "Beach in Mirissa", "Tea plantations in Nuwara Eliya"]
preferred_activities = ["hiking", "swimming", "cultural tours"]

In [31]:
user_features = preprocess_user_preferences(bucket_list, preferred_activities, feature_extraction_results)

Starting user preferences preprocessing...
User preferences preprocessing completed.


In [33]:
user_features.shape

(1, 1355)

In [34]:
import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec

def save_feature_extraction_results(feature_extraction_results, directory='saved_models'):
    print("Saving feature extraction results...")

    # Create directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)

    # Save individual components
    joblib.dump(feature_extraction_results['tfidf_vectorizer'], os.path.join(directory, 'tfidf_vectorizer.joblib'))
    joblib.dump(feature_extraction_results['svd_model'], os.path.join(directory, 'svd_model.joblib'))
    feature_extraction_results['word2vec_model'].save(os.path.join(directory, 'word2vec_model'))
    np.save(os.path.join(directory, 'sentence_embeddings.npy'), feature_extraction_results['sentence_embeddings'])
    np.save(os.path.join(directory, 'valid_indices.npy'), feature_extraction_results['valid_indices'])

    # Save feature_df separately as it might be large
    feature_extraction_results['feature_df'].to_parquet(os.path.join(directory, 'feature_df.parquet'))

    # Save tfidf_matrix as a sparse matrix
    joblib.dump(feature_extraction_results['tfidf_matrix'], os.path.join(directory, 'tfidf_matrix.joblib'))

    print("Feature extraction results saved successfully.")

In [35]:

def load_feature_extraction_results(directory='saved_models'):
    print("Loading feature extraction results...")

    feature_extraction_results = {}

    # Load individual components
    feature_extraction_results['tfidf_vectorizer'] = joblib.load(os.path.join(directory, 'tfidf_vectorizer.joblib'))
    feature_extraction_results['svd_model'] = joblib.load(os.path.join(directory, 'svd_model.joblib'))
    feature_extraction_results['word2vec_model'] = Word2Vec.load(os.path.join(directory, 'word2vec_model'))
    feature_extraction_results['sentence_embeddings'] = np.load(os.path.join(directory, 'sentence_embeddings.npy'))
    feature_extraction_results['valid_indices'] = np.load(os.path.join(directory, 'valid_indices.npy'))

    # Load feature_df
    feature_extraction_results['feature_df'] = pd.read_parquet(os.path.join(directory, 'feature_df.parquet'))

    # Load tfidf_matrix
    feature_extraction_results['tfidf_matrix'] = joblib.load(os.path.join(directory, 'tfidf_matrix.joblib'))

    print("Feature extraction results loaded successfully.")
    return feature_extraction_results

In [36]:
# Save results
save_feature_extraction_results(feature_extraction_results)

# Load results
# loaded_results = load_feature_extraction_results()

Saving feature extraction results...
Feature extraction results saved successfully.
